In [28]:
import os
import glob
import flywheel
import logging
import zipfile
import platform
import subprocess
import shutil

In [29]:
#Initialize the logger
logging.basicConfig(filename='download_mrs.log', level=logging.INFO)
logger = logging.getLogger()

In [30]:
# Get API Key from local environment variables and initialize the Flywheel client
#api_key = os.environ.get('FW_KEY')
api_key = 'cuny-mri.flywheel.io:djE0mwC43qLLZTOw09aNwL7FKMRA7y_mwx7U0negVGflQ0XzSzFvmiSNA'
fw = flywheel.Client(api_key)

In [34]:
# Path where the MRS data will be downloaded
root_path = '/Users/jacekdmochowski/PROJECTS/fus/data/bold_flywheel/'

# Get the Flywheel Project
project = fw.lookup('jacek/FUSRelabeledCopy')

In [37]:
import flywheel
import os
from pprint import pprint

def explore_project_structure(project_label):
    """
    Explore and print the structure of a Flywheel project to locate analyses.
    
    Parameters:
    -----------
    project_label : str
        Name of the Flywheel project
    """
    #fw = flywheel.Client()
    #project = fw.projects.find_first('label="{}"'.format(project_label))
    
    if not project:
        raise ValueError(f"Project '{project_label}' not found")
    
    print(f"\nProject: {project.label} ({project.id})")
    print(f"Number of sessions: {len(list(project.sessions()))}")
    
    # Explore first session in detail
    first_session = list(project.sessions())[0]
    print(f"\nFirst Session Details:")
    print(f"Label: {first_session.label}")
    print(f"ID: {first_session.id}")
    
    # Check for analyses at different levels
    print("\nAnalyses at different levels:")
    
    # Project level
    project_analyses = list(project.analyses)
    print(f"\nProject-level analyses: {len(project_analyses)}")
    if project_analyses:
        print("First project analysis details:")
        pprint(vars(project_analyses[0]))
    
    # Session level (trying different methods)
    print(f"\nSession-level analyses:")
    try:
        # Try direct attribute
        session_analyses = first_session.analyses
        print(f"Via .analyses attribute: {len(session_analyses) if session_analyses else 'None'}")
    except Exception as e:
        print(f"Error accessing .analyses: {str(e)}")
        
    try:
        # Try get_analyses() method
        session_analyses = list(first_session.get_analyses())
        print(f"Via get_analyses(): {len(session_analyses)}")
        if session_analyses:
            print("First session analysis details:")
            pprint(vars(session_analyses[0]))
    except Exception as e:
        print(f"Error accessing get_analyses(): {str(e)}")
    
    # Acquisition level
    print("\nAcquisitions in first session:")
    acquisitions = list(first_session.acquisitions())
    print(f"Number of acquisitions: {len(acquisitions)}")
    
    if acquisitions:
        first_acq = acquisitions[0]
        print(f"\nFirst acquisition details:")
        print(f"Label: {first_acq.label}")
        try:
            acq_analyses = list(first_acq.analyses)
            print(f"Number of analyses: {len(acq_analyses)}")
            if acq_analyses:
                print("First acquisition analysis details:")
                pprint(vars(acq_analyses[0]))
        except Exception as e:
            print(f"Error accessing acquisition analyses: {str(e)}")

def find_analyses_recursive(project_label):
    """
    Recursively search for analyses containing BOLD files at all levels.
    
    Parameters:
    -----------
    project_label : str
        Name of the Flywheel project
    """
    #fw = flywheel.Client()
    #project = fw.projects.find_first('label="{}"'.format(project_label))
    
    if not project:
        raise ValueError(f"Project '{project_label}' not found")
    
    found_analyses = []
    
    # Check project level
    for analysis in project.analyses:
        for file_ in analysis.files:
            if 'resampled_bold' in file_.name and file_.name.endswith('.nii.gz'):
                found_analyses.append((analysis, file_))
    
    # Check each session
    for session in project.sessions():
        # Check session level
        try:
            for analysis in session.analyses:
                for file_ in analysis.files:
                    if 'resampled_bold' in file_.name and file_.name.endswith('.nii.gz'):
                        found_analyses.append((analysis, file_))
        except Exception:
            pass
            
        # Check each acquisition
        for acquisition in session.acquisitions():
            try:
                for analysis in acquisition.analyses:
                    for file_ in analysis.files:
                        if 'resampled_bold' in file_.name and file_.name.endswith('.nii.gz'):
                            found_analyses.append((analysis, file_))
            except Exception:
                pass
    
    print(f"\nFound {len(found_analyses)} analyses with matching BOLD files:")
    for analysis, file_ in found_analyses:
        print(f"\nAnalysis ID: {analysis.id}")
        print(f"File name: {file_.name}")
        print(f"Parent container: {analysis.parent.container_type}")
        print(f"Parent ID: {analysis.parent.id}")


In [36]:
PROJECT_NAME = "jacek/FUSRelabeledCopy"
#    
print("Exploring project structure...")
explore_project_structure("")

## Example usage
#if __name__ == "__main__":
#    PROJECT_NAME = "your-project-name"
#    
#    print("Exploring project structure...")
#    explore_project_structure(PROJECT_NAME)
    
#    print("\nSearching for analyses with BOLD files...")
#    find_analyses_recursive(PROJECT_NAME)

Exploring project structure...

Project: FUSRelabeledCopy (66c36f6b50d2146c7067429f)
Number of sessions: 53

First Session Details:
Label: NOSTIM
ID: 66c36facc63c9958181cb5ce

Analyses at different levels:

Project-level analyses: 0

Session-level analyses:
Via .analyses attribute: None
Error accessing get_analyses(): type object 'object' has no attribute 'get_analyses'

Acquisitions in first session:
Number of acquisitions: 14

First acquisition details:
Label: anat-scout_run-01_ignore_BIDS
Number of analyses: 0


In [39]:
find_analyses_recursive("")


Found 0 analyses with matching BOLD files:


In [5]:
fus_project = project.reload()
for analysis in project.analyses:
    print('%s: %s' % (analysis.id, analysis.label))

In [5]:
def download_all_mrs(root_data):
    #Get the group label
    group_label = project.group
    
    #Iterate through all the files in the 'ACTIVE' and 'SHAM' sessions in the project
    for subject in project.subjects():
        
        for session in subject.sessions():
            
            # Input the session labels with MRS data.
            
            if session.label == 'ACTIVE' or session.label == 'SHAM': 
                
                for acquisition in session.acquisitions():
                    
                    acq_label = acquisition.label
                    
                    #Check if the acquisition label starts with 'mrs'
                    
                    if acq_label.startswith('mrs'):
                        #Create the directory using the Flywheel hierarchy
                        path = f'{root_path}/{group_label}/{project.label}/{subject.label}/{session.label}/{acq_label}'
                        os.makedirs(path, exist_ok=True)
                        
                        for file in acquisition.files:
                            
                            #Find the dicom file in the mrs acquisition
                            if file.type == 'dicom':
                                
                                file_path = f'{path}/{file.name}'
                                        
                                file_name = f'sub-{subject.label}_ses-{session.label}_acq-{acq_label}'
                                
                                dcm_path = f'{path}/{file_name}'

                                if not os.path.exists(dcm_path):
                                    if platform.system() == 'Windows':
                                        file_path = file_path.replace('/', '\\')
                                        dcm_path = dcm_path.replace('/', '\\')
                                    try:
                                        #Download the file from the acquisition.
                                        
                                        acquisition.download_file(file.name, file_path)
                                        
                                        # Open the dicom zip file
                                        with zipfile.ZipFile(file_path, 'r') as zip_ref:
                                            #Extract the contents of the zip file to the path
                                            zip_ref.extractall(path)
                                            
                                        #Delete dicom compressed zip file
                                        os.remove(file_path)
                                        
                                        #Rename the unzipped folder to the new file name
                                        unzipped_folder = glob.glob(f'{path}/*')[0]
                                        os.rename(unzipped_folder, dcm_path)

                                        logger.info(f'Downloaded {file.name} from {acq_label} in {session.label} for {subject.label}')
                                        
                                        # convert dicoms to nifti
                                        subprocess.run(["spec2nii", "dicom", "-f", dcm_path, dcm_path])
                                        
                                        #Remove the dicoms
                                        shutil.rmtree(dcm_path)
                                        
                                        logger.info(f'Converted dicom into nifti: {file.name} from {acq_label} in {session.label} for {subject.label}')
                                        
                                    except Exception as e:
                                        logger.error(f'Error downloading {file.name} from {acq_label} in {session.label} for {subject.label}. {e}')
                                break

In [6]:
download_all_mrs(root_path)

/opt/miniconda3/envs/fus/lib/python3.13/site-packages/spec2nii/Siemens/dicomfunctions.py:10: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  import nibabel.nicom.dicomwrappers


Found 3 files.


/opt/miniconda3/envs/fus/lib/python3.13/site-packages/spec2nii/Siemens/dicomfunctions.py:10: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  import nibabel.nicom.dicomwrappers


Found 3 files.


/opt/miniconda3/envs/fus/lib/python3.13/site-packages/spec2nii/Siemens/dicomfunctions.py:10: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  import nibabel.nicom.dicomwrappers


...

/opt/miniconda3/envs/fus/lib/python3.13/site-packages/spec2nii/Siemens/dicomfunctions.py:10: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  import nibabel.nicom.dicomwrappers


Found 3 files.


/opt/miniconda3/envs/fus/lib/python3.13/site-packages/spec2nii/Siemens/dicomfunctions.py:10: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  import nibabel.nicom.dicomwrappers


Found 3 files.
Found 3 files.


/opt/miniconda3/envs/fus/lib/python3.13/site-packages/spec2nii/Siemens/dicomfunctions.py:10: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  import nibabel.nicom.dicomwrappers
